# Clipboard Health Shift Offers Analysis

Analysis of shift offers data for Clipboard Health. The goal of this analysis is to look for signal in the marketplace and identify interesting patterns in the data.

### Description for Clipboard Health
> Clipboard health is a two sided marketplace with strong network effects where workers transact with workplaces to book per diem shifts in the future. Workplaces post shifts and workers choose which shifts they’d like to work by booking them.

## Data Loading / Initial Data Exploration

In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta

# Set seaborn style
plt.style.use('fivethirtyeight')
sns.set_palette('RdYlBu')

# Load the data
csv_path = "../data/Problems we tackle, Shift Offers v3 - table_12_2025-01-22T1134.csv"
df = pd.read_csv(csv_path)

# Display the first few rows
df.head()

,SHIFT_ID,WORKER_ID,WORKPLACE_ID,SHIFT_START_AT,SHIFT_CREATED_AT,OFFER_VIEWED_AT,DURATION,SLOT,CLAIMED_AT,DELETED_AT,IS_VERIFIED,CANCELED_AT,IS_NCNS,PAY_RATE,CHARGE_RATE
0,6757580b1e2d97752fd69167,65b01f2e46c0645699081cbe,5e7e45243bfbb200165914ae,2024-12-09 23:00:00,2024-12-09 20:50:19,2024-12-09 21:18:42,8,pm,NaN,NaN,False,NaN,False,21.29,29
1,675d37d8a1ca6192a74d23f4,65298a18cc967a5cebbd40b6,5e1ce78827ff480016e9133e,2024-12-14 22:30:00,2024-12-14 7:46:32,2024-12-14 13:19:30,9,pm,NaN,2024-12-14 19:23:43,False,NaN,False,23.23,30
2,67550bddd79613f860549322,6696d1c1d0200bf317ee5d3c,626b0b89596c0601c2c39642,2024-12-08 15:00:00,2024-12-08 3:00:46,2024-12-08 4:04:14,6,am,NaN,NaN,False,NaN,False,21.97,30
3,66f5d05de01fd3697b18c206,66b285d5d0200bf317738e59,5cb9f07135163900163f532c,2024-09-27 14:00:00,2024-09-26 21:21:34,2024-09-27 4:19:45,8,am,NaN,NaN,False,NaN,False,19.05,28
4,66ee3848e62bb5f43e3baee5,620c6429e2ceb601ad203920,611af67795f4c501662edb31,2024-10-08 21:30:00,2024-09-21 3:06:48,2024-10-06 0:46:37,8,pm,NaN,NaN,False,NaN,False,22.13,24


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 266340 entries, 0 to 266339
Data columns (total 15 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   SHIFT_ID          266340 non-null  object 
 1   WORKER_ID         266340 non-null  object 
 2   WORKPLACE_ID      266340 non-null  object 
 3   SHIFT_START_AT    266340 non-null  object 
 4   SHIFT_CREATED_AT  266340 non-null  object 
 5   OFFER_VIEWED_AT   266340 non-null  object 
 6   DURATION          266340 non-null  int64  
 7   SLOT              266340 non-null  object 
 8   CLAIMED_AT        13064 non-null   object 
 9   DELETED_AT        55644 non-null   object 
 10  IS_VERIFIED       266340 non-null  bool   
 11  CANCELED_AT       321 non-null     object 
 12  IS_NCNS           266340 non-null  bool   
 13  PAY_RATE          266340 non-null  float64
 14  CHARGE_RATE       266340 non-null  int64  
dtypes: bool(2), float64(1), int64(2), object(10)
memory usage: 26.9+ MB


### Data Cleanup

In [9]:
# Convert datetime columns to pandas datetime objects
datetime_cols = ['SHIFT_START_AT', 'SHIFT_CREATED_AT', 'OFFER_VIEWED_AT', 'CLAIMED_AT', 'CANCELED_AT', 'DELETED_AT']
for col in datetime_cols:
    if col in df.columns:
        df[col] = pd.to_datetime(df[col], errors='coerce')

In [10]:
def calc_time_diff_col(col1, col2, secs_per_unit=60):
    """
    Calculate the time difference between two datetime columns in days.
    """
    if col1 in df.columns and col2 in df.columns:
        return (df[col2] - df[col1]).dt.total_seconds() / secs_per_unit
    else:
        return None

# Time between shift creation and start
df['creation_to_start_days'] = calc_time_diff_col('SHIFT_CREATED_AT', 'SHIFT_START_AT', secs_per_unit=60*60*24)

# Time between viewing and claiming (for claimed shifts)
df['view_to_claim_minutes'] = calc_time_diff_col('OFFER_VIEWED_AT', 'CLAIMED_AT')

# Time between claiming and canceling (for canceled shifts)
df['claim_to_cancel_hours'] = calc_time_diff_col('CLAIMED_AT', 'CANCELED_AT', secs_per_unit=60*60)

# Time between shift creation and viewing
df['creation_to_view_hours'] = calc_time_diff_col('SHIFT_CREATED_AT', 'OFFER_VIEWED_AT', secs_per_unit=60*60)

In [11]:
# Create a flag for weekend shifts
df['is_weekend'] = df['SHIFT_START_AT'].dt.dayofweek >= 5  # 5 and 6 are Saturday and Sunday

# Calculate revenue per shift offer (charge rate - pay rate) * duration
if 'charge rate' in df.columns:
    df['revenue_per_shift'] = (df['charge rate'] - df['PAY_RATE']) * df['DURATION']

# Create a flag for shifts that were claimed
df['was_claimed'] = df['CLAIMED_AT'].notnull()

# Create a flag for claimed shifts that were canceled
df['was_canceled'] = df['CANCELED_AT'].notnull() & df['was_claimed']

# Create a flag for shifts that were deleted by workplace
df['was_deleted'] = df['DELETED_AT'].notnull()

In [13]:
# Calculate key marketplace metrics
total_shift_offers = len(df)
total_unique_shifts = df['SHIFT_ID'].nunique()
total_unique_workers = df['WORKER_ID'].nunique()
total_unique_workplaces = df['WORKPLACE_ID'].nunique()

# Calculate claim rate
claim_rate = df['was_claimed'].mean() * 100

# Calculate cancellation rate (out of claimed shifts)
cancellation_rate = df[df['was_claimed']]['was_canceled'].mean() * 100 if len(df[df['was_claimed']]) > 0 else 0

# Calculate no-show rate
no_show_rate = df[df['was_claimed']]['IS_NCNS'].mean() * 100 if len(df[df['was_claimed']]) > 0 else 0

# Calculate deletion rate of claimed shifts
deletion_rate = df[df['was_claimed']]['was_deleted'].mean() * 100 if len(df[df['was_claimed']]) > 0 else 0

# Calculate completion rate
completion_rate = df[df['was_claimed']]['IS_VERIFIED'].mean() * 100 if len(df[df['was_claimed']]) > 0 else 0

# Calculate average pay rate and duration
avg_pay_rate = df['PAY_RATE'].mean()
avg_duration = df['DURATION'].mean()

# Print summary metrics
print(f"Total Shift Offers: {total_shift_offers:,}")
print(f"Total Unique Shifts: {total_unique_shifts:,}")
print(f"Total Unique Workers: {total_unique_workers:,}")
print(f"Total Unique Workplaces: {total_unique_workplaces:,}")
print(f"Claim Rate: {claim_rate:.2f}%")
print(f"Cancellation Rate (of claimed shifts): {cancellation_rate:.2f}%")
print(f"No-Show Rate (of claimed shifts): {no_show_rate:.2f}%")
print(f"Deletion Rate (of claimed shifts): {deletion_rate:.2f}%")
print(f"Completion Rate (of claimed shifts): {completion_rate:.2f}%")
print(f"Average Pay Rate: ${avg_pay_rate:.2f}/hour")
print(f"Average Shift Duration: {avg_duration:.2f} hours")

Total Shift Offers: 266,340
Total Unique Shifts: 19,900
Total Unique Workers: 10,291
Total Unique Workplaces: 132
Claim Rate: 4.91%
Cancellation Rate (of claimed shifts): 2.46%
No-Show Rate (of claimed shifts): 0.24%
Deletion Rate (of claimed shifts): 1.51%
Completion Rate (of claimed shifts): 96.82%
Average Pay Rate: $24.16/hour
Average Shift Duration: 8.34 hours
